<a href="https://colab.research.google.com/github/putrinahampun/PreparationForTFDSExam/blob/main/ExerciseDicoding/SubmissionB/ProblemB3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [3]:
# ========================================================================================
# PROBLEM B3
#
# Build a CNN based classifier for Rock-Paper-Scissors dataset.
# Your input layer should accept 150x150 with 3 bytes color as the input shape.
# This is unlabeled data, use ImageDataGenerator to automatically label it.
# Don't use lambda layers in your model.
#
# The dataset used in this problem is created by Laurence Moroney (laurencemoroney.com).
#
# Desired accuracy AND validation_accuracy > 83%
# ========================================================================================

import urllib.request
import zipfile
import tensorflow as tf
import os
from keras.preprocessing.image import ImageDataGenerator


def solution_B3():
    data_url = 'https://github.com/dicodingacademy/assets/releases/download/release-rps/rps.zip'
    urllib.request.urlretrieve(data_url, 'rps.zip')
    local_file = 'rps.zip'
    zip_ref = zipfile.ZipFile(local_file, 'r')
    zip_ref.extractall('data/')
    zip_ref.close()

    data_url2 = 'https://storage.googleapis.com/tensorflow-1-public/course2/week4/rps-test-set.zip'
    urllib.request.urlretrieve(data_url2, 'rps-test-set.zip')
    local_file2 = 'rps-test-set.zip'
    zip_ref2 = zipfile.ZipFile(local_file2, 'r')
    zip_ref2.extractall('data/')
    zip_ref2.close()

    TRAINING_DIR = "data/rps/"
    VALIDATION_DIR = "data/rps-test-set"

    training_datagen = ImageDataGenerator(
        # YOUR CODE HERE
        rescale=1./255,
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest'
    )

    validation_datagen = ImageDataGenerator(rescale=1./255)

    # YOUR IMAGE SIZE SHOULD BE 150x150
    # Make sure you used "categorical"
    train_generator = training_datagen.flow_from_directory(
        # YOUR CODE
        TRAINING_DIR,
        target_size=(150, 150),
        class_mode='categorical',
        batch_size=126
    )

    validation_generator = validation_datagen.flow_from_directory(
        VALIDATION_DIR,
        target_size=(150, 150),
        class_mode='categorical',
        batch_size=126
    )

    model=tf.keras.models.Sequential([
    # YOUR CODE HERE, end with 3 Neuron Dense, activated by softmax
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu', input_shape=(150, 150, 3)),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(64, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),
        tf.keras.layers.Conv2D(128, (3, 3), activation='relu'),
        tf.keras.layers.MaxPooling2D(2, 2),

        tf.keras.layers.Flatten(),
        tf.keras.layers.Dropout(0.5),
        tf.keras.layers.Dense(512, activation='relu'),
        tf.keras.layers.Dense(3, activation='softmax')
    ])

    model.compile(
        loss='categorical_crossentropy',
        optimizer='rmsprop',
        metrics=['accuracy'])

    class myCallback(tf.keras.callbacks.Callback):
      def on_epoch_end(self, epoch, logs={}):
        '''
        Halts the training when the loss falls below 0.4

        Args:
          epoch (integer) - index of epoch (required but unused in the function definition below)
          logs (dict) - metric results from the training epoch
        '''

        # Check the loss
        if(logs.get('accuracy') > 0.85):

          # Stop if threshold is met
          print("\nAccuracy achieve more than 83% so cancelling training!")
          self.model.stop_training = True

    # Instantiate class
    callbacks = myCallback()
    history = model.fit(
        train_generator,
        epochs=15,
        steps_per_epoch=20,
        validation_data=validation_generator,
        verbose=1,
        validation_steps=3,
        callbacks = [callbacks])
    return model


# The code below is to save your model as a .h5 file.
# It will be saved automatically in your Submission folder.
if __name__ == '__main__':
    # DO NOT CHANGE THIS CODE
    model=solution_B3()
    model.save("model_B3.h5")


Found 2520 images belonging to 3 classes.
Found 372 images belonging to 3 classes.
Epoch 1/15
20/20 [==============================] - 213s 11s/step - loss: 1.1631 - accuracy: 0.3468 - val_loss: 1.0957 - val_accuracy: 0.3333
Epoch 2/15
20/20 [==============================] - 206s 10s/step - loss: 1.0968 - accuracy: 0.3552 - val_loss: 1.0827 - val_accuracy: 0.3333
Epoch 3/15
20/20 [==============================] - 211s 10s/step - loss: 1.0925 - accuracy: 0.4210 - val_loss: 1.1602 - val_accuracy: 0.3333
Epoch 4/15
20/20 [==============================] - 208s 10s/step - loss: 1.0963 - accuracy: 0.4456 - val_loss: 1.0583 - val_accuracy: 0.5780
Epoch 5/15
20/20 [==============================] - 207s 10s/step - loss: 0.9990 - accuracy: 0.5190 - val_loss: 0.9648 - val_accuracy: 0.5269
Epoch 6/15
20/20 [==============================] - 216s 11s/step - loss: 0.8643 - accuracy: 0.5857 - val_loss: 0.7065 - val_accuracy: 0.6720
Epoch 7/15
20/20 [==============================] - 207s 10s/step

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3000: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(
